In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark =  SparkSession.builder.getOrCreate()

In [42]:
jugadores = spark.read.option('header','true').option('inferSchema','true').csv('../../data/practica/players.csv')

games = spark.read.option('header','true').option('inferSchema','true').csv('../../data/practica/games.csv')

apariciones = spark.read.option('header','true').option('inferSchema','true').csv('../../data/practica/appearances.csv')

ligas = spark.read.option('header','true').option('inferSchema','true').csv('../../data/practica/leagues.csv')


In [44]:
games.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- competition_code: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: string (nullable = true)
 |-- date: date (nullable = true)
 |-- home_club_id: integer (nullable = true)
 |-- away_club_id: integer (nullable = true)
 |-- home_club_goals: integer (nullable = true)
 |-- away_club_goals: integer (nullable = true)
 |-- home_club_position: integer (nullable = true)
 |-- away_club_position: integer (nullable = true)
 |-- stadium: string (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- referee: string (nullable = true)
 |-- url: string (nullable = true)



In [18]:
from pyspark.sql.functions import col,desc,count
jugadores.groupBy('country_of_birth').count().orderBy(desc('count')).filter(col('country_of_birth').isNotNull()).limit(3).show()

+----------------+-----+
|country_of_birth|count|
+----------------+-----+
|          France| 1694|
|           Spain| 1388|
|           Italy| 1312|
+----------------+-----+



In [37]:
#Obtenga la lista de jugadores con tarjeta roja. La salida debe contener dos columnas, el nombre de pila del jugador y la cantidad de tarjetas rojas que tiene.
#apariciones.printSchema()
#jugadores.printSchema()

j = jugadores.alias("j")
a = apariciones.alias("a")

tarjetas_rojas = j.join(a, col('j.player_id') == col('a.player_id'))

tarjetas_rojas.printSchema()



root
 |-- player_id: integer (nullable = true)
 |-- current_club_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- position: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- market_value_in_gbp: double (nullable = true)
 |-- highest_market_value_in_gbp: double (nullable = true)
 |-- url: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- game_id: integer (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: integer (nullable = true)
 |-- goals: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- minutes_played: integer (nullable = true)
 |-- yellow_cards: integer

In [41]:
tarjetas_rojas.groupBy('a.player_id').agg(count('red_cards').alias('TARJETAS_ROJAS')).orderBy(desc('TARJETAS_ROJAS')).limit(10).show()

+---------+--------------+
|player_id|TARJETAS_ROJAS|
+---------+--------------+
|    32467|           377|
|   125781|           375|
|    28003|           366|
|    65230|           365|
|    74229|           350|
|    38253|           350|
|    58358|           347|
|    18922|           344|
|     8198|           343|
|    59561|           341|
+---------+--------------+



In [43]:
# ¿Cuántos juegos se jugaron en la Premier League? La salida debe contener dos columnas, el nombre de la liga y la cantidad de juegos que se jugaron en ella.
ligas.printSchema()

root
 |-- league_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- confederation: string (nullable = true)

